## 0. データセットについて
###  概要
- Kaggle "House Rent Prediction Dataset"
- https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset


### 場所
- 本ノートブックの「input」フォルダ＞「263」内に存在 ※263はデータID
- KAMONOHASHI上で作成したデータセットを読み込んだ場合、inputフォルダ内に配置されます

## 1. データセット読み込み
### csv読み込み

In [1]:
import cudf #cuDF(pandasに対応するライブラリ)
import cuml #cuML(scikit-learnに対応するライブラリ)

In [2]:
#import pandas as pd
import datetime as dt
import xgboost as xgb
from xgboost import dask as dxgb

In [3]:
file_path = '../input/training/263/House_Rent_Dataset.csv' #データセットの場所

#pdf = pd.read_csv(file_path) #pandas dataframe
cdf = cudf.read_csv(file_path) #cudf dataframe

In [4]:
cdf #データセット表示

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner
...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2022-05-18,2,15000,1000,3 out of 5,Carpet Area,Bandam Kommu,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner
4742,2022-05-15,3,29000,2000,1 out of 4,Super Area,"Manikonda, Hyderabad",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner
4743,2022-07-10,3,35000,1750,3 out of 5,Carpet Area,"Himayath Nagar, NH 7",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent
4744,2022-07-06,3,45000,1500,23 out of 34,Carpet Area,Gachibowli,Hyderabad,Semi-Furnished,Family,2,Contact Agent


## 2. 前処理

### 説明変数(x), 目的変数(y)に分割
- 目的変数：Rent
- 説明変数：Rent, Posted On以外の変数

In [5]:
cdf.columns #カラム名表示

Index(['Posted On', 'BHK', 'Rent', 'Size', 'Floor', 'Area Type',
       'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred',
       'Bathroom', 'Point of Contact'],
      dtype='object')

In [6]:
X_cdf = cdf.iloc[:, 1:]
X_cdf = X_cdf.drop(['Rent'], axis=1)
y_cdf = cdf.iloc[:, 2]

In [7]:
X_cdf

,BHK,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner
...,...,...,...,...,...,...,...,...,...,...
4741,2,1000,3 out of 5,Carpet Area,Bandam Kommu,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner
4742,3,2000,1 out of 4,Super Area,"Manikonda, Hyderabad",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner
4743,3,1750,3 out of 5,Carpet Area,"Himayath Nagar, NH 7",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent
4744,3,1500,23 out of 34,Carpet Area,Gachibowli,Hyderabad,Semi-Furnished,Family,2,Contact Agent


In [8]:
y_cdf

0       10000
1       20000
2       17000
3       10000
4        7500
        ...  
4741    15000
4742    29000
4743    35000
4744    45000
4745    15000
Name: Rent, Length: 4746, dtype: int64

### 特徴量変換

In [9]:
_columns = ['Floor', 'Area Type', 'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred', 'Point of Contact']

X_cdf = cudf.get_dummies(X_cdf, columns=_columns) #カテゴリ変数を1-hotベクトルに変換

## 3. GPUクラスターの設定
- 実装コードはこちらの記事(https://qiita.com/shin_ishiguro/items/8f39aac45acc8363a42e) を参考にさせていただきました
- 本ノートブックではGPUコア数を1に設定しているため、作成されるWorkerの数は1つとなります

In [10]:
import dask
import dask_cudf
import xgboost
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess

In [11]:
cmd = "hostname --all-ip-addresses"
process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
IPADDR = str(output.decode()).split()[0]

cluster = LocalCUDACluster(ip=IPADDR)
client = Client(cluster) #processes=False)#
client

2023-09-30 11:46:19,208 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-09-30 11:46:19,208 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://10.233.96.59:8787/status,
Dashboard: http://10.233.96.59:8787/status,Workers: 1
Total threads: 1,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://10.233.96.59:34887,Workers: 1
Dashboard: http://10.233.96.59:8787/status,Total threads: 1
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://10.233.96.59:40399,Total threads: 1
Dashboard: http://10.233.96.59:36895/status,Memory: 7.45 GiB
Nanny: tcp://10.233.96.59:40681,


## 4. dataframeの変換
- pandas dataframeからdask dataframeに変換を行う
- データ変換することで初めて、Multi GPUでの機械学習が可能になる

In [12]:
from cuml.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_cdf, y_cdf, train_size=0.8, random_state=2023) #データセットを学習用とテスト用に分割

In [14]:
X_train.head()

,BHK,Size,Bathroom,Floor_1,Floor_1 out of 1,Floor_1 out of 10,Floor_1 out of 11,Floor_1 out of 12,Floor_1 out of 13,Floor_1 out of 14,...,City_Mumbai,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Point of Contact_Contact Agent,Point of Contact_Contact Builder,Point of Contact_Contact Owner
1564,1,450,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
4497,3,1700,3,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
66,1,300,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
2164,1,500,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
3850,2,700,3,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1


In [15]:
y_train.head()

1564    12000
4497    25000
66       5000
2164    10000
3850     9000
Name: Rent, dtype: int64

In [16]:
X_test.head()

,BHK,Size,Bathroom,Floor_1,Floor_1 out of 1,Floor_1 out of 10,Floor_1 out of 11,Floor_1 out of 12,Floor_1 out of 13,Floor_1 out of 14,...,City_Mumbai,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Point of Contact_Contact Agent,Point of Contact_Contact Builder,Point of Contact_Contact Owner
534,1,480,2,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,1,0,0
2611,3,900,2,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1,0,0
3863,2,1600,2,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
1539,1,600,2,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
1546,1,800,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1


In [17]:
y_test.head()

534     33000
2611    31000
3863    26000
1539    10000
1546     9000
Name: Rent, dtype: int64

In [18]:
#cudf dataframe -> dask dataframe
X_train_ddf = dask_cudf.from_cudf(X_train, npartitions=1)
X_test_ddf = dask_cudf.from_cudf(X_test, npartitions=1)
y_train_ddf = dask_cudf.from_cudf(y_train, npartitions=1)
y_test_ddf = dask_cudf.from_cudf(y_test, npartitions=1)

In [19]:
X_train_ddf

<dask_cudf.DataFrame | 1 tasks | 1 npartitions>

In [20]:
X_train_ddf.head()

,BHK,Size,Bathroom,Floor_1,Floor_1 out of 1,Floor_1 out of 10,Floor_1 out of 11,Floor_1 out of 12,Floor_1 out of 13,Floor_1 out of 14,...,City_Mumbai,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Point of Contact_Contact Agent,Point of Contact_Contact Builder,Point of Contact_Contact Owner
1,2,800,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
2,2,1000,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
3,2,800,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
4,2,850,1,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
5,2,600,2,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1


In [21]:
y_train_ddf.head()

1    20000
2    17000
3    10000
4     7500
5     7000
Name: Rent, dtype: int64

## 5. XGBoostの学習、実行

In [22]:
params = {
    'max_depth': 8, 
    'max_leaves': 200, 
    'tree_method': 'gpu_hist',
    'objective': 'reg:squarederror',
    'eval_metric': 'auc',
    'grow_policy': 'lossguide'
}

In [23]:
dtrain = xgb.dask.DaskDMatrix(client, X_train_ddf, y_train_ddf)

In [24]:
#GPUクラスターとパラメータを指定して学習
model = xgb.dask.train(client, params, dtrain, num_boost_round=300)

/opt/conda/envs/rapids/lib/python3.8/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[11:46:27] task [xgboost.dask]:tcp://10.233.96.59:40399 got new rank 0


## 6. 予測と評価

### 予測

In [25]:
pred = xgb.dask.predict(client, model, X_test_ddf).compute()

In [26]:
pred

0       13810.885742
6       13323.054688
14      14965.730469
20       6185.766113
27      11613.211914
            ...     
4726    25411.064453
4727    66146.695312
4736    20705.148438
4740    15224.536133
4741    14295.231445
Name: 0, Length: 950, dtype: float32

In [27]:
y_test

534     33000
2611    31000
3863    26000
1539    10000
1546     9000
        ...  
4740    12000
2001    10000
4503    25000
3729    40000
1777    16000
Name: Rent, Length: 950, dtype: int64

In [28]:
#予測値のデータが自動でソートされているので、正解データもソートしておく
y_test2 = y_test.sort_index()
y_test2

0       10000
6       10000
14       8500
20       7900
27       6000
        ...  
4726    16000
4727    30000
4736    15000
4740    12000
4741    15000
Name: Rent, Length: 950, dtype: int64

In [29]:
print(type(pred))
print(type(y_test2))

<class 'cudf.core.series.Series'>
<class 'cudf.core.series.Series'>


### 評価

In [30]:
from cuml.metrics import roc_auc_score

In [31]:
roc_auc_score(y_test2, pred)

0.4999920129776001